## Model testing on prepared data

In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('./../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.multicovariate_models.gmm_functions import predict_groups_gmm, fit_gaussian_mixture_model
from src.multicovariate_models.GroupsModel import GroupsModel

In [ ]:
%%time

main_df = pd.read_csv("simulated_sensor_data_8_cols.csv")
main_df = main_df.drop(columns=["origin_file"]).drop_duplicates()
display(main_df)

In [ ]:
%%time

# sklearn GroupShuffleSplit and pandas.cut may come in handy for splitting data sometime
model_class = GroupsModel(model_type="GMM", prepared_data_df=main_df)
# x_train, x_test, y_train, y_test = model_class.get_data()

In [ ]:
%%time

model_class.train()